<a href="https://colab.research.google.com/github/Maneekul/Intern/blob/main/THcars_S_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  import tarfile
  import os
  import itertools
  import matplotlib.pylab as plt
  import numpy as np
  import tensorflow as tf
  import tensorflow_hub as hub
  import csv
  import time
  import torch
  import json
  from random import randint
  from pathlib import Path
  from PIL import Image
  import PIL.Image
  from scipy.io import loadmat
  import pandas as pd
  from torch import nn
  from torch import optim
  import torch.nn.functional as F
  from torchvision import datasets, transforms, models
  import torchvision.models as models
  from matplotlib.ticker import FormatStrFormatter
  from google.colab.patches import cv2_imshow
  from google.colab import drive
  import matplotlib.image as mpimg
  from matplotlib.pyplot import imshow
  from IPython.display import Image
%matplotlib inline


In [ ]:
from PIL import Image

def crop_image(image_path):
    img = Image.open(image_path)

    # Get the size of the image
    width, height = img.size

    # Set the threshold value for the image
    threshold = 10

    # Find the top edge of the image
    for y in range(height):
        for x in range(width):
            if img.getpixel((x,y)) != (255, 255, 255):
                top = y
                break
        else:
            continue
        break

    # Find the bottom edge of the image
    for y in range(height-1, 0, -1):
        for x in range(width):
            if img.getpixel((x,y)) != (255, 255, 255):
                bottom = y
                break
        else:
            continue
        break

    # Find the left edge of the image
    for x in range(width):
        for y in range(height):
            if img.getpixel((x,y)) != (255, 255, 255):
                left = x
                break
        else:
            continue
        break

    # Find the right edge of the image
    for x in range(width-1, 0, -1):
        for y in range(height):
            if img.getpixel((x,y)) != (255, 255, 255):
                right = x
                break
        else:
            continue
        break

    # Crop the image and save it
    img.crop((left-threshold, top-threshold, right+threshold, bottom+threshold)).save('cropped_image.png')

path = '/content/drive/MyDrive/CARStest'
files = os.listdir(path)

for file in files:
    if file.endswith('.jpg'):

      img_path = os.path.join(path, file)
      print(img_path)
      mycrop = crop_image(img_path)



#path = '/content/drive/MyDrive/CARStest'
#files = os.listdir(path)
#
#for file in files:
#    if file.endswith('.jpg'):
#        img_path = os.path.join(path, file)

/content/drive/MyDrive/CARStest/IMG_6197.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_5.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_27.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_32.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_44.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_47.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_49.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_59.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_83.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_87.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_99.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_101.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_103.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_23.jpg
/content/drive/MyDrive/CARStest/LINE_ALBUM_Da-5Apr_๒๓๐๔๐๕_15.jpg
/content/drive/MyDrive/CARStest/20230418_165

In [ ]:
print('TF version:', tf.__version__)
print('Hub version:', hub.__version__)
print('Phsical devices:', tf.config.list_physical_devices())

def get_hub_url_and_isize(model_name, ckpt_type, hub_type):
  if ckpt_type == '1k':
    ckpt_type = ''  # json doesn't support empty string
  else:
    ckpt_type = '-' + ckpt_type  # add '-' as prefix

  hub_url_map = {
    'efficientnetv2-b0': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/{hub_type}',
    'efficientnetv2-b1': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1/{hub_type}',
    'efficientnetv2-b2': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2/{hub_type}',
    'efficientnetv2-b3': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3/{hub_type}',
    'efficientnetv2-s':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s/{hub_type}',
    'efficientnetv2-m':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m/{hub_type}',
    'efficientnetv2-l':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l/{hub_type}',

    'efficientnetv2-b0-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k/{hub_type}',
    'efficientnetv2-b1-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k/{hub_type}',
    'efficientnetv2-b2-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k/{hub_type}',
    'efficientnetv2-b3-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k/{hub_type}',
    'efficientnetv2-s-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k/{hub_type}',
    'efficientnetv2-m-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k/{hub_type}',
    'efficientnetv2-l-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k/{hub_type}',
    'efficientnetv2-xl-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k/{hub_type}',

    'efficientnetv2-b0-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k-ft1k/{hub_type}',
    'efficientnetv2-b1-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k-ft1k/{hub_type}',
    'efficientnetv2-b2-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k-ft1k/{hub_type}',
    'efficientnetv2-b3-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k-ft1k/{hub_type}',
    'efficientnetv2-s-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k-ft1k/{hub_type}',
    'efficientnetv2-m-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k-ft1k/{hub_type}',
    'efficientnetv2-l-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k-ft1k/{hub_type}',
    'efficientnetv2-xl-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k-ft1k/{hub_type}',

    # efficientnetv1
    'efficientnet_b0': f'https://tfhub.dev/tensorflow/efficientnet/b0/{hub_type}/1',
    'efficientnet_b1': f'https://tfhub.dev/tensorflow/efficientnet/b1/{hub_type}/1',
    'efficientnet_b2': f'https://tfhub.dev/tensorflow/efficientnet/b2/{hub_type}/1',
    'efficientnet_b3': f'https://tfhub.dev/tensorflow/efficientnet/b3/{hub_type}/1',
    'efficientnet_b4': f'https://tfhub.dev/tensorflow/efficientnet/b4/{hub_type}/1',
    'efficientnet_b5': f'https://tfhub.dev/tensorflow/efficientnet/b5/{hub_type}/1',
    'efficientnet_b6': f'https://tfhub.dev/tensorflow/efficientnet/b6/{hub_type}/1',
    'efficientnet_b7': f'https://tfhub.dev/tensorflow/efficientnet/b7/{hub_type}/1',
  }

  image_size_map = {
    'efficientnetv2-b0': 224,
    'efficientnetv2-b1': 240,
    'efficientnetv2-b2': 260,
    'efficientnetv2-b3': 300,
    'efficientnetv2-s':  384,
    'efficientnetv2-m':  480,
    'efficientnetv2-l':  480,
    'efficientnetv2-xl':  512,

    'efficientnet_b0': 224,
    'efficientnet_b1': 240,
    'efficientnet_b2': 260,
    'efficientnet_b3': 300,
    'efficientnet_b4': 380,
    'efficientnet_b5': 456,
    'efficientnet_b6': 528,
    'efficientnet_b7': 600,
  }

  hub_url = hub_url_map.get(model_name + ckpt_type)
  image_size = image_size_map.get(model_name, 224)
  return hub_url, image_size


def get_imagenet_labels(filename):
  labels = []
  with open(filename, 'r') as f:
    for line in f:
      labels.append(line.split('\t')[1][:-1])  # split and remove line break.
  return labels


TF version: 2.12.0
Hub version: 0.13.0
Phsical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Build your model
import tensorflow_hub as hub
model_name = 'efficientnetv2-s' #@param {type:'string'}
ckpt_type = '1k'   # @param ['21k', '21k-ft1k', '1k']
hub_type = 'classification' # @param ['feature-vector'] classification
batch_size =  8 #@param {type:"integer"}
hub_url, image_size = get_hub_url_and_isize(model_name, ckpt_type, hub_type)


In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=(image_size, image_size),
                       batch_size=batch_size,
                       interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)

valid_generator = valid_datagen.flow_from_directory(
   '/content/drive/MyDrive/CarsTH', subset="validation", shuffle=False, **dataflow_kwargs)


do_data_augmentation = False #@param {type:"boolean"}
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/CarsTH', subset="training", shuffle=True, **dataflow_kwargs)
print('train_generator',train_generator)


train_image_names = train_generator.filenames
valid_image_names = valid_generator.filenames

print("Training image names: ", train_image_names)
print("Validation image names: ", valid_image_names)

Found 68 images belonging to 6 classes.
Found 281 images belonging to 6 classes.
train_generator <keras.preprocessing.image.DirectoryIterator object at 0x7fe2eb793a30>
Training image names:  ['ALTIST/IMG_2353.JPG', 'ALTIST/IMG_2436.JPG', 'ALTIST/IMG_2448.JPG', 'ALTIST/IMG_2449.JPG', 'ALTIST/IMG_2451.JPG', 'ALTIST/IMG_2459.JPG', 'ALTIST/IMG_2615.JPG', 'ALTIST/IMG_2817.JPG', 'ALTIST/IMG_2827 - Copy.JPG', 'ALTIST/IMG_2827.JPG', 'ALTIST/IMG_2861.JPG', 'ALTIST/IMG_2948.JPG', 'ALTIST/IMG_2949 - Copy.JPG', 'ALTIST/IMG_2949.JPG', 'ALTIST/IMG_2954.JPG', 'ALTIST/IMG_2959 - Copy.JPG', 'ALTIST/IMG_2959.JPG', 'ALTIST/IMG_2960.JPG', 'ALTIST/IMG_2961.JPG', 'ALTIST/IMG_2969 - Copy.JPG', 'ALTIST/IMG_2969.JPG', 'ALTIST/IMG_2971.JPG', 'ALTIST/IMG_2983.JPG', 'ALTIST/IMG_3085.JPG', 'ALTIST/IMG_3097.JPG', 'ALTIST/IMG_6433.jpg', 'ALTIST/IMG_6449.jpg', 'ALTIST/IMG_6478.jpg', 'ALTIST/LINE_ALBUM_Da-5Apr_41.jpg', 'ALTIST/LINE_ALBUM_Da-5Apr_46.jpg', 'ALTIST/LINE_ALBUM_Da-5Apr_53.jpg', 'ALTIST/LINE_ALBUM_Da-5Apr_5

In [ ]:
# whether to finetune the whole model or just the top layer.
do_fine_tuning = True #@param {type:"boolean"}
num_epochs = 5 #@param {type:"integer"}

tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=[image_size, image_size, 3]),
    hub.KerasLayer(hub_url, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None, image_size, image_size, 3))
model.summary()

model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
hist = model.fit(
    train_generator,
    epochs=num_epochs, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps).history

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1000)              21612360  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 6)                 6006      
                                                                 
Total params: 21,618,366
Trainable params: 21,464,494
Non-trainable params: 153,872
_________________________________________________________________
Epoch 1/5
35/35 [==============================] - 226s 5s/step - loss: 1.9489 - accuracy: 0.2747 - val_loss: 2.0038 - val_accuracy: 0.2500
Epoch 2/5
35/35 [==============================] - 82s 2s/step - loss: 1.0819 - accuracy: 0.6777 - val_loss: 2.1537 - val_accuracy: 0.3750
Epoch 3/5
35/35 [======

In [ ]:

data_test_cars = pd.read_csv('/content/drive/MyDrive/csv_label/label_CarsTH_test.csv', header=None, names=[ 'filename', 'class','minx'], encoding='ISO-8859-1')
class_test = data_test_cars.values.tolist()
print(class_test)
print(class_test[0][0])
print(class_test[0][1])
print(class_test[0][2])





[['20230405_101442.jpg', 1, 'ALTIST'], ['20230420_091520.jpg', 1, 'ALTIST'], ['IMG_2450.JPG', 1, 'ALTIST'], ['IMG_2550.JPG', 1, 'ALTIST'], ['IMG_2970.JPG', 1, 'ALTIST'], ['IMG_2980.JPG', 1, 'ALTIST'], ['LINE_ALBUM_Da-5Apr_òóðôðõ_44.jpg', 1, 'ALTIST'], ['LINE_ALBUM_Da-5Apr_òóðôðõ_47.jpg', 1, 'ALTIST'], ['20230417_172549.jpg', 2, 'CITY'], ['20230418_165811.jpg', 2, 'CITY'], ['20230418_170130.jpg', 2, 'CITY'], ['20230418_171348.jpg', 2, 'CITY'], ['20230420_091443.jpg', 2, 'CITY'], ['20230420_121244.jpg', 2, 'CITY'], ['20230424_120148.jpg', 2, 'CITY'], ['IMG_2359.JPG', 2, 'CITY'], ['IMG_2388.JPG', 2, 'CITY'], ['IMG_2418.JPG', 2, 'CITY'], ['IMG_2503.JPG', 2, 'CITY'], ['IMG_2523.JPG', 2, 'CITY'], ['IMG_2622.JPG', 2, 'CITY'], ['IMG_2665.JPG', 2, 'CITY'], ['IMG_2839.JPG', 2, 'CITY'], ['IMG_2928.JPG', 2, 'CITY'], ['20230418_171131.jpg', 3, 'CIVIC'], ['20230418_171255.jpg', 3, 'CIVIC'], ['20230420_091426.jpg', 3, 'CIVIC'], ['IMG_2334.JPG', 3, 'CIVIC'], ['IMG_2348.JPG', 3, 'CIVIC'], ['IMG_2415.JP

In [ ]:
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    print(classes)
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx
classes, idx = find_classes("/content/drive/MyDrive/CarTH_test")


view = pd.DataFrame(classes, columns=['class'])
print(view.to_string(index=False))
print(classes[0])

['ALTIST', 'CITY', 'Civic', 'Mazda2', 'VIOS', 'Yaris']
 class
ALTIST
  CITY
 Civic
Mazda2
  VIOS
 Yaris
ALTIST


In [ ]:

def get_class_string_from_index(index):
    for class_string, class_index in valid_generator.class_indices.items():
        if class_index == index:
            return class_string


path = '/content/drive/MyDrive/CARStest'
files = os.listdir(path)

top1_count = 0
top2_count = 0
top3_count = 0
top4_count = 0
top5_count = 0
AllWrong = 0

for file in files:
    if file.endswith('.jpg'):
        img_path = os.path.join(path, file)
        img = PIL.Image.open(img_path)

        ## Convert the image to a 4-channel image
        #img = img.convert('RGBA')
        img = img.resize((384, 384))
        img = np.array(img)
        img = img[:, :, :3]
        img = img / 255.0
        img = np.expand_dims(img, axis=0)

        #print(img.shape)
        picture_label = file.split('.')[0]  # Get the true label from the filename
        picture_label = picture_label  + '.jpg'
        print(picture_label)

    for i in range(len(class_test)):
      if class_test[i][0] == picture_label:
        x = class_test[i][1]
        x = int(float(x))
        print('class number =',x)
        true_label = classes[x-1]
        print('true_label = ',true_label)

        top_k = 5
        prediction_scores = model.predict(img)
        predicted_indices = (-prediction_scores[0]).argsort()[:top_k]
        predicted_labels = [get_class_string_from_index(index) for index in predicted_indices]
        print(predicted_labels)

        print(predicted_labels[0])
        print(predicted_labels[1])
        print(predicted_labels[2])
        print(predicted_labels[3])
        print(predicted_labels[4])


        if predicted_labels[0] == true_label:
            top1_count += 1
            print('11111111111111111111111111111111111111111111111111111111111111111111111111111')
        elif predicted_labels[1] == true_label:
            top2_count += 1
            print('22222222222222222222222222222222222222222222222222222222222222222222222222222')
        elif predicted_labels[2] == true_label:
            top3_count += 1
            print('33333333333333333333333333333333333333333333333333333333333333333333333333333')
        elif predicted_labels[3] == true_label:
            top4_count += 1
            print('444444444444444444444444444444444444444444444444444444444444444444444444444444')
        elif predicted_labels[4] == true_label:
            top5_count += 1
            print('555555555555555555555555555555555555555555555555555555555555555555555555555555')
        else:
            AllWrong += 1
            print("ALL SQUENCE WRONGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG")
print("\n")
print("Top 1 accuracy: ", top1_count/len(files))
print("Top 2 accuracy: ", top2_count/len(files))
print("Top 3 accuracy: ", top3_count/len(files))
print("Top 4 accuracy: ", top4_count/len(files))
print("Top 5 accuracy: ", top5_count/len(files))
print("  ALL WRONG   : ", AllWrong/len(files))



IMG_6197.jpg
class number = 6
true_label =  Yaris
1/1 [==============================] - 1s 1s/step
['ALTIST', 'CITY', 'Mazda2', 'VIOS', 'Yaris']
ALTIST
CITY
Mazda2
VIOS
Yaris
555555555555555555555555555555555555555555555555555555555555555555555555555555
class number = 6
true_label =  Yaris
1/1 [==============================] - 0s 31ms/step
['ALTIST', 'CITY', 'Mazda2', 'VIOS', 'Yaris']
ALTIST
CITY
Mazda2
VIOS
Yaris
555555555555555555555555555555555555555555555555555555555555555555555555555555
class number = 6
true_label =  Yaris
1/1 [==============================] - 0s 32ms/step
['ALTIST', 'CITY', 'Mazda2', 'VIOS', 'Yaris']
ALTIST
CITY
Mazda2
VIOS
Yaris
555555555555555555555555555555555555555555555555555555555555555555555555555555
class number = 6
true_label =  Yaris
1/1 [==============================] - 0s 37ms/step
['ALTIST', 'CITY', 'Mazda2', 'VIOS', 'Yaris']
ALTIST
CITY
Mazda2
VIOS
Yaris
555555555555555555555555555555555555555555555555555555555555555555555555555555
class number 